# 分類：target を正解データとして、画像の数値を判断する

ライブラリのインポート

In [3]:
import pandas as pd
#文字化けの防止
%matplotlib inline
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor

CSVファイルのインポート CSVファイルを読み込んで、先頭５行表示

In [4]:
df = pd.read_csv('digits.csv')
#df.head(5)

#左端の行番号削除
df2 = df.drop(df.columns[[0, 0]], axis=1)
#df2.head(5)

☆カテゴリ特徴量列無し(ダミー変数化処理無し)

正解データの確認

In [5]:
#正解データ(target)の種類の確認 - 不均衡データの確認
df2['target'].value_counts()

3    183
1    182
5    182
4    181
6    181
9    180
7    179
0    178
2    177
8    174
Name: target, dtype: int64

☆欠損値の処理 - 欠損値無し

In [6]:
#欠損値の確認
df2.isnull().sum()

#欠損値無し

pixel_0_0    0
pixel_0_1    0
pixel_0_2    0
pixel_0_3    0
pixel_0_4    0
            ..
pixel_7_4    0
pixel_7_5    0
pixel_7_6    0
pixel_7_7    0
target       0
Length: 65, dtype: int64

☆外れ値の処理 - 決定木は外れ値の処理無し

訓練データ＆検証データとテストデータに分離


In [7]:
#訓練データ&検証データとテストデータに分割
train_val, test = train_test_split(df2, test_size=0.2, random_state=0)

☆特徴量を取り出し標準化 - ばらつきが無いので標準化しない。正解データも取り出す

In [8]:
#データの種類
df2.value_counts()

pixel_0_0  pixel_0_1  pixel_0_2  pixel_0_3  pixel_0_4  pixel_0_5  pixel_0_6  pixel_0_7  pixel_1_0  pixel_1_1  pixel_1_2  pixel_1_3  pixel_1_4  pixel_1_5  pixel_1_6  pixel_1_7  pixel_2_0  pixel_2_1  pixel_2_2  pixel_2_3  pixel_2_4  pixel_2_5  pixel_2_6  pixel_2_7  pixel_3_0  pixel_3_1  pixel_3_2  pixel_3_3  pixel_3_4  pixel_3_5  pixel_3_6  pixel_3_7  pixel_4_0  pixel_4_1  pixel_4_2  pixel_4_3  pixel_4_4  pixel_4_5  pixel_4_6  pixel_4_7  pixel_5_0  pixel_5_1  pixel_5_2  pixel_5_3  pixel_5_4  pixel_5_5  pixel_5_6  pixel_5_7  pixel_6_0  pixel_6_1  pixel_6_2  pixel_6_3  pixel_6_4  pixel_6_5  pixel_6_6  pixel_6_7  pixel_7_0  pixel_7_1  pixel_7_2  pixel_7_3  pixel_7_4  pixel_7_5  pixel_7_6  pixel_7_7  target
0          0          0          0          3          14         3          0          0          0          0          1          14         16         5          0          0          1          9          15         16         16         4          0          0          4          12 

In [9]:
#特徴量と正解データの取り出し
x = train_val.drop(['target'], axis=1) #特徴量
t = train_val[['target']] #正解

#特徴量は標準化 - なし
#sc = StandardScaler()
#sc_x = sc.fit_transform(x)

訓練データと検証データに分離

In [10]:
#訓練データと検証データに分離
x_train, x_test, y_train, y_test = train_test_split(x, t, test_size=0.2, random_state=0)

In [11]:
#確認用
#x.head(5)
#t.head(5)
#print(x_train)

★1 決定木の場合

In [12]:
#決定木モデルの作成と学習
#訓練データ, 検証データ, 木の深さを引数とし、訓練データと検証データの正解率とモデルを返す
def dtree_Learn(x_train, x_test, y_train, y_test, depth=3):
  model = tree.DecisionTreeClassifier(max_depth=depth, random_state=0, class_weight='balanced')
  model.fit(x_train, y_train) #学習

  score1 = model.score(x_train, y_train)
  score2 = model.score(x_test, y_test)
  return round(score1, 3), round(score2, 3), model

In [13]:
for j in range(1,15): #jは木の深さ
  train_score, test_score, model = dtree_Learn(x_train, x_test, y_train, y_test, j)
  ret_str = '深さ{}: 訓練データの正解率{}  検証データの正解率{}'
  print(ret_str.format(j, train_score,test_score))

深さ1: 訓練データの正解率0.205  検証データの正解率0.205
深さ2: 訓練データの正解率0.322  検証データの正解率0.299
深さ3: 訓練データの正解率0.488  検証データの正解率0.469
深さ4: 訓練データの正解率0.604  検証データの正解率0.594
深さ5: 訓練データの正解率0.708  検証データの正解率0.708
深さ6: 訓練データの正解率0.833  検証データの正解率0.757
深さ7: 訓練データの正解率0.901  検証データの正解率0.802
深さ8: 訓練データの正解率0.938  検証データの正解率0.823
深さ9: 訓練データの正解率0.97  検証データの正解率0.83
深さ10: 訓練データの正解率0.984  検証データの正解率0.823
深さ11: 訓練データの正解率0.995  検証データの正解率0.837
深さ12: 訓練データの正解率0.999  検証データの正解率0.837
深さ13: 訓練データの正解率1.0  検証データの正解率0.837
深さ14: 訓練データの正解率1.0  検証データの正解率0.837


In [14]:
#検証データの成果率から深さは11、深さ11の決定木モデルを作成
dtree_train_score, dtree_test_score, dtree_model = dtree_Learn(x_train, x_test, y_train, y_test, 11)
ret_str = '深さ{}: 訓練データの正解率{}  検証データの正解率{}'
print(ret_str.format(11, dtree_train_score,dtree_test_score))

深さ11: 訓練データの正解率0.995  検証データの正解率0.837


★2 ロジスティック回帰の場合

In [15]:
#ロジスティック回帰モデルの作成と学習
#訓練データ, 検証データ, 正則化項の定数を引数とし、訓練データと検証データの正解率とモデルを返す
def logreg_Learn(x_train, x_test, y_train, y_test, wk=0.1):
  model = LogisticRegression(random_state=0, C=wk, multi_class='auto', solver='lbfgs')
  model.fit(x_train, y_train) #学習

  score1 = model.score(x_train, y_train)
  score2 = model.score(x_test, y_test)
  return round(score1, 3), round(score2, 3), model

In [16]:
#ロジスティック回帰による学習
#正則化項の定数を0.01 ～ 0.1まで0.01刻みで検証
for i in range(1, 11):
  num = i/100

  train_score, test_score, model = logreg_Learn(x_train, x_test, y_train, y_test, num)
  ret_str = '正則化の影響力を調整する重み{}: 訓練データの正解率{}  検証データの正解率{}'
  print(ret_str.format(num, train_score, test_score))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example

正則化の影響力を調整する重み0.01: 訓練データの正解率0.991  検証データの正解率0.983


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


正則化の影響力を調整する重み0.02: 訓練データの正解率0.998  検証データの正解率0.983


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


正則化の影響力を調整する重み0.08: 訓練データの正解率0.999  検証データの正解率0.979


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


正則化の影響力を調整する重み0.09: 訓練データの正解率0.999  検証データの正解率0.979
正則化の影響力を調整する重み0.1: 訓練データの正解率1.0  検証データの正解率0.979


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [17]:
#正則化の影響力を調整する重み0.02: 訓練データの正解率0.998  検証データの正解率0.983
#重み0.02のロジスティック回帰モデルを作成
logreg_train_score, logreg_test_score, logreg_model = logreg_Learn(x_train, x_test, y_train, y_test, 0.02)
ret_str = '正則化の影響力を調整する重み{}: 訓練データの正解率{}  検証データの正解率{}'
print(ret_str.format(0.02, logreg_train_score, logreg_test_score))


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


正則化の影響力を調整する重み0.02: 訓練データの正解率0.998  検証データの正解率0.983


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


★3 ランダムフォレストの場合

In [18]:
#ランダムフォレストの作成と学習
#訓練データ, 検証データ, 正則化項の定数を引数とし、訓練データと検証データの正解率とモデルを返す
def rndfst_Learn(x_train, x_test, y_train, y_test, wk=200):
  model = RandomForestClassifier(n_estimators=wk, random_state=0)
  model.fit(x_train, y_train) #学習

  score1 = model.score(x_train, y_train)
  score2 = model.score(x_test, y_test)
  return round(score1, 3), round(score2, 3), model

In [19]:
#ランダムフォレストの木の個数を確認
#木の個数を100 ～ 500まで100刻みで検証
for i in range(100, 501, 100):
  train_score, test_score, model = rndfst_Learn(x_train, x_test, y_train, y_test, i)
  ret_str = '木の個数 {}: 訓練データの正解率{}  検証データの正解率{}'
  print(ret_str.format(i, train_score, test_score))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


木の個数 100: 訓練データの正解率1.0  検証データの正解率0.99


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


木の個数 200: 訓練データの正解率1.0  検証データの正解率0.979


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


木の個数 300: 訓練データの正解率1.0  検証データの正解率0.983


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


木の個数 400: 訓練データの正解率1.0  検証データの正解率0.983


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


木の個数 500: 訓練データの正解率1.0  検証データの正解率0.983


In [20]:
#木の個数100: 訓練データの正解率1.0  検証データの正解率0.99 以降変、0.979と0.983の繰り返し
#木の個数100でランダムフォレストを作成
rndfst_train_score, rndfst_test_score, rndfst_model = rndfst_Learn(x_train, x_test, y_train, y_test, 100)
ret_str = '木の個数 {}: 訓練データの正解率{}  検証データの正解率{}'
print(ret_str.format(100, rndfst_train_score, rndfst_test_score))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


木の個数 100: 訓練データの正解率1.0  検証データの正解率0.99


★4 アダブーストの場合

In [21]:
#アダブーストの作成と学習
#訓練データ, 検証データ, 決定木の個数を引数とし、訓練データと検証データの正解率とモデルを返す
def adbst_Learn(x_train, x_test, y_train, y_test, wk=500):
  #★1の結果より 深さ11の決定木を作成
  base_model = tree.DecisionTreeClassifier(max_depth=11, random_state=0)

  model = AdaBoostClassifier(n_estimators=wk, random_state=0, base_estimator=base_model)
  model.fit(x_train, y_train) #学習

  score1 = model.score(x_train, y_train)
  score2 = model.score(x_test, y_test)
  return round(score1, 3), round(score2, 3), model

In [22]:
#アダブーストで使用する決定木の個数を確認
#木の個数を300 ～ 700まで100刻みで検証
for i in range(100, 701, 100):
  train_score, test_score, model = adbst_Learn(x_train, x_test, y_train, y_test, i)
  ret_str = '決定木の個数 {}: 訓練データの正解率{}  検証データの正解率{}'
  print(ret_str.format(i, train_score, test_score))

決定木の個数 100: 訓練データの正解率1.0  検証データの正解率0.903
決定木の個数 200: 訓練データの正解率1.0  検証データの正解率0.903


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


決定木の個数 300: 訓練データの正解率1.0  検証データの正解率0.903
決定木の個数 400: 訓練データの正解率1.0  検証データの正解率0.903
決定木の個数 500: 訓練データの正解率1.0  検証データの正解率0.903


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


決定木の個数 600: 訓練データの正解率1.0  検証データの正解率0.903
決定木の個数 700: 訓練データの正解率1.0  検証データの正解率0.903


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [23]:
#決定木の個数を100から増やしても、訓練データの正解率1.0  検証データの正解率0.903 以降、変わらない
#決定木の個数100でアダブーストを作成
adbst_train_score, adbst_test_score, adbst_model = adbst_Learn(x_train, x_test, y_train, y_test, 100)
ret_str = '決定木の個数 {}: 訓練データの正解率{}  検証データの正解率{}'
print(ret_str.format(100, adbst_train_score, adbst_test_score))

決定木の個数 100: 訓練データの正解率1.0  検証データの正解率0.903


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


★5 ランダムフォレスト回帰の場合

In [24]:
#ランダムフォレスト回帰の作成と学習
#訓練データ, 検証データ, 正則化項の定数を引数とし、訓練データと検証データの正解率とモデルを返す
def rndfst_reg_Learn(x_train, x_test, y_train, y_test, wk=200):
  model = RandomForestRegressor(n_estimators=wk, random_state=0)
  model.fit(x_train, y_train) #学習

  score1 = model.score(x_train, y_train)
  score2 = model.score(x_test, y_test)
  return round(score1, 3), round(score2, 3), model

In [25]:
#ランダムフォレスト回帰の木の個数を確認
#木の個数を100 ～ 500まで100刻みで検証
for i in range(100, 501, 100):
  train_score, test_score, model = rndfst_reg_Learn(x_train, x_test, y_train, y_test, i)
  ret_str = '木の個数 {}: 訓練データの正解率{}  検証データの正解率{}'
  print(ret_str.format(i, train_score, test_score))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


木の個数 100: 訓練データの正解率0.98  検証データの正解率0.876


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


木の個数 200: 訓練データの正解率0.98  検証データの正解率0.872


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


木の個数 300: 訓練データの正解率0.981  検証データの正解率0.872


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


木の個数 400: 訓練データの正解率0.981  検証データの正解率0.872


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


木の個数 500: 訓練データの正解率0.981  検証データの正解率0.873


In [26]:
#木の個数100: 訓練データの正解率0.98  検証データの正解率0.876 - 検証データの正解率が良い
#木の個数100でランダムフォレスト回帰を作成
rndfst_reg_train_score, rndfst_reg_test_score, rndfst_reg_model = rndfst_reg_Learn(x_train, x_test, y_train, y_test, 100)
ret_str = '木の個数 {}: 訓練データの正解率{}  検証データの正解率{}'
print(ret_str.format(100, rndfst_reg_train_score, rndfst_reg_test_score))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


木の個数 100: 訓練データの正解率0.98  検証データの正解率0.876


★6 アダブースト回帰の場合

In [27]:
#アダブースト回帰の作成と学習
#訓練データ, 検証データ, 決定木の個数を引数とし、訓練データと検証データの正解率とモデルを返す
def adbst_reg_Learn(x_train, x_test, y_train, y_test, wk=100):
  #★1の結果より 深さ11の決定木を作成
  base_model = tree.DecisionTreeRegressor(max_depth=11, random_state=0)

  model = AdaBoostRegressor(n_estimators=wk, random_state=0, base_estimator=base_model)
  model.fit(x_train, y_train) #学習

  score1 = model.score(x_train, y_train)
  score2 = model.score(x_test, y_test)
  return round(score1, 3), round(score2, 3), model

In [28]:
#アダブースト回帰で使用する決定木の個数を確認
#木の個数を100 ～ 500まで100刻みで検証
for i in range(100, 501, 100):
  train_score, test_score, model = adbst_reg_Learn(x_train, x_test, y_train, y_test, i)
  ret_str = '決定木の個数 {}: 訓練データの正解率{}  検証データの正解率{}'
  print(ret_str.format(i, train_score, test_score))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


決定木の個数 100: 訓練データの正解率1.0  検証データの正解率0.978


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


決定木の個数 200: 訓練データの正解率1.0  検証データの正解率0.977


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


決定木の個数 300: 訓練データの正解率1.0  検証データの正解率0.977


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


決定木の個数 400: 訓練データの正解率1.0  検証データの正解率0.976


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


決定木の個数 500: 訓練データの正解率1.0  検証データの正解率0.978


In [29]:
#上記結果より決定木の個数100でアダブースト回帰を作成
adbst_reg_train_score, adbst_reg_test_score, adbst_reg_model = adbst_reg_Learn(x_train, x_test, y_train, y_test, 100)
ret_str = '決定木の個数 {}: 訓練データの正解率{}  検証データの正解率{}'
print(ret_str.format(100, adbst_reg_train_score, adbst_reg_test_score))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


決定木の個数 100: 訓練データの正解率1.0  検証データの正解率0.978


モデルの選択 - 下記結果より、ランダムフォレスト が良いと思われる

In [30]:
#比較
print('決定木                ： 訓練データの正解率{}  検証データの正解率{}'.format(dtree_train_score,dtree_test_score))    #dtree_model
print('ロジスティック回帰    ： 訓練データの正解率{}  検証データの正解率{}'.format(logreg_train_score,logreg_test_score))  #logreg_model
print('ランダムフォレスト    ： 訓練データの正解率{}  検証データの正解率{}'.format(rndfst_train_score, rndfst_test_score)) #rndfst_model
print('アダブースト          ： 訓練データの正解率{}  検証データの正解率{}'.format(adbst_train_score, adbst_test_score))   #adbst_model
print('ランダムフォレスト回帰： 訓練データの正解率{}  検証データの正解率{}'.format(rndfst_reg_train_score, rndfst_reg_test_score)) #rndfst_reg_model
print('アダブースト回帰      ： 訓練データの正解率{}  検証データの正解率{}'.format(adbst_reg_train_score, adbst_reg_test_score))   #adbst_reg_model

決定木                ： 訓練データの正解率0.995  検証データの正解率0.837
ロジスティック回帰    ： 訓練データの正解率0.998  検証データの正解率0.983
ランダムフォレスト    ： 訓練データの正解率1.0  検証データの正解率0.99
アダブースト          ： 訓練データの正解率1.0  検証データの正解率0.903
ランダムフォレスト回帰： 訓練データの正解率0.98  検証データの正解率0.876
アダブースト回帰      ： 訓練データの正解率1.0  検証データの正解率0.978


正解率または決定係数の表示（testデータを使用する)

In [31]:
#testデータを特徴量と正解データに分割 - 標準化無し
test_x = test.drop(['target'], axis=1)    #特徴量
test_t = test[['target']] #正解

In [42]:
#比較
msg_str = '訓練データ{:.3f}  検証データ{:.3f}  テストデータ{:.3f}'
print('決定木                ：', msg_str.format(dtree_train_score, dtree_test_score, dtree_model.score(test_x, test_t)))    #dtree_model
print('ロジスティック回帰    ：' ,msg_str.format(logreg_train_score, logreg_test_score, logreg_model.score(test_x, test_t)))  #logreg_model
print('ランダムフォレスト★  ：' ,msg_str.format(rndfst_train_score, rndfst_test_score, rndfst_model.score(test_x, test_t))) #rndfst_model
print('アダブースト          ：' ,msg_str.format(adbst_train_score, adbst_test_score, adbst_model.score(test_x, test_t)))   #adbst_model
print('ランダムフォレスト回帰：' ,msg_str.format(rndfst_reg_train_score, rndfst_reg_test_score, rndfst_reg_model.score(test_x, test_t))) #rndfst_reg_model
print('アダブースト回帰      ：' ,msg_str.format(adbst_reg_train_score, adbst_reg_test_score, adbst_reg_model.score(test_x, test_t)))   #adbst_reg_model

決定木                ： 訓練データ0.995  検証データ0.837  テストデータ0.814
ロジスティック回帰    ： 訓練データ0.998  検証データ0.983  テストデータ0.961
ランダムフォレスト★  ： 訓練データ1.000  検証データ0.990  テストデータ0.975
アダブースト          ： 訓練データ1.000  検証データ0.903  テストデータ0.903
ランダムフォレスト回帰： 訓練データ0.980  検証データ0.876  テストデータ0.867
アダブースト回帰      ： 訓練データ1.000  検証データ0.978  テストデータ0.943


保存：モデルは digits.pkl, 選択モデルはランダムフォレスト

In [43]:
import pickle

with open('digits.pkl', 'wb') as f:
  pickle.dump(rndfst_model, f)